In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import os
import time
import numpy as np
import copy


# Hyper Parameters
num_epochs = 10
batch_size = 125
learning_rate = 0.001

# Assign to GPU if gpu is available
use_gpu = torch.cuda.is_available()

# Set up 
data_transforms = {
    'train': transforms.Compose([transforms.RandomSizedCrop(24),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor()]),
    'test': transforms.Compose([transforms.RandomSizedCrop(24),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor()
                                ]), }

# Read in the train and test datasets
data_dir = './data'
dsets = {x: dsets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'test']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=True, num_workers=8)
                for x in ['train', 'test']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'test']}
print(dset_sizes)
dset_classes = dsets['train'].classes

# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(3*3*256, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
        
model = CNN()
model = model.cuda()


# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

since = time.time()

best_model = model
best_acc = 0.0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'test']:
        if phase == 'train':
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            model.train(True)  # Set model to training mode
        else:
            model.train(False)  # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for data in dset_loaders[phase]:
            # get the inputs
            inputs, labels = data

            # wrap them in Variable
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            # backward + optimize only if in training phase
            if phase == 'train':
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dset_sizes[phase]
        epoch_acc = running_corrects / dset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f} Img/sec: {:.2f}'.format(
              phase, epoch_loss, epoch_acc, 
            (dset_sizes[phase] * (epoch + 1)/(time.time() - since))))

        # deep copy the model
        if phase == 'test' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model)

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# Save the Trained Model
torch.save(best_model.state_dict(), './model.pth')

{'test': 10000, 'train': 50000}
Epoch 0/9
----------
train Loss: 0.0165 Acc: 0.2104 Img/sec: 6537.57
test Loss: 0.0147 Acc: 0.3118 Img/sec: 1184.54
Epoch 1/9
----------
train Loss: 0.0141 Acc: 0.3444 Img/sec: 6515.64
test Loss: 0.0134 Acc: 0.3799 Img/sec: 1237.03
Epoch 2/9
----------
train Loss: 0.0130 Acc: 0.3968 Img/sec: 6492.72
test Loss: 0.0125 Acc: 0.4286 Img/sec: 1254.38
Epoch 3/9
----------
train Loss: 0.0120 Acc: 0.4544 Img/sec: 6478.87
test Loss: 0.0116 Acc: 0.4770 Img/sec: 1262.39
Epoch 4/9
----------
train Loss: 0.0113 Acc: 0.4913 Img/sec: 6465.77
test Loss: 0.0108 Acc: 0.5148 Img/sec: 1265.85
Epoch 5/9
----------
train Loss: 0.0107 Acc: 0.5197 Img/sec: 6453.83
test Loss: 0.0112 Acc: 0.5068 Img/sec: 1268.77
Epoch 6/9
----------
train Loss: 0.0103 Acc: 0.5414 Img/sec: 6443.35
test Loss: 0.0104 Acc: 0.5355 Img/sec: 1270.04
Epoch 7/9
----------
train Loss: 0.0099 Acc: 0.5570 Img/sec: 6434.48
test Loss: 0.0100 Acc: 0.5589 Img/sec: 1270.25
Epoch 8/9
----------
train Loss: 0.0097 